<a href="https://colab.research.google.com/github/meti-94/BERT-QA/blob/main/BuboQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fuzzywuzzy
!pip uninstall -y torch
!pip install torch==0.4.1
!pip install torchtext==0.2.3

Uninstalling torch-1.8.1+cu101:
  Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 519.5MB 34kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 0.4.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 0.4.1 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for torchtext: filename=torchtext-0.2.3-cp37-none-any.whl size=40134 sha256=bfa5cbb9f75c2745f00e80fcc1677e45c8c6c25c69d9efd34294bee80b051e9e
  Stored in directory: /root/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [11]:
!git clone https://github.com/castorini/BuboQA.git
%cd BuboQA

Cloning into 'BuboQA'...
remote: Enumerating objects: 1383, done.
remote: Total 1383 (delta 0), reused 0 (delta 0), pack-reused 1383
Receiving objects: 100% (1383/1383), 361.07 KiB | 4.40 MiB/s, done.
Resolving deltas: 100% (908/908), done.
/content/BuboQA


In [32]:
%cd /content/BuboQA
!bash setup.sh 

/content/BuboQA
Running fetch_dataset script

--2021-04-01 09:44:21--  https://git.uwaterloo.ca/jimmylin/BuboQA-data/raw/master/SimpleQuestions_v2.tgz
Resolving git.uwaterloo.ca (git.uwaterloo.ca)... 129.97.83.4
Connecting to git.uwaterloo.ca (git.uwaterloo.ca)|129.97.83.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 423435590 (404M) [application/x-gzip]
Saving to: ‘SimpleQuestions_v2.tgz’

SimpleQuestions_v2. 100%[===================>] 403.82M  29.7MB/s    in 14s     

2021-04-01 09:44:40 (28.2 MB/s) - ‘SimpleQuestions_v2.tgz’ saved [423435590/423435590]



Unzipping SimpleQuestions dataset...

tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
./._SimpleQuestions_v2
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended head

In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-04-01 08:46:05--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-01 08:46:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-01 08:46:06--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [3]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [4]:
import pandas as pd
from collections import Counter
from tqdm import tqdm
from sklearn.preprocessing import LabelBinarizer
from copy import copy
import numpy as np
from sklearn.linear_model import LogisticRegression

In [5]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.300d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [45]:
train = pd.read_excel('/content/drive/MyDrive/QA/train.xlsx')
valid = pd.read_excel('/content/drive/MyDrive/QA/valid.xlsx')
test = pd.read_excel('/content/drive/MyDrive/QA/test.xlsx')

In [46]:
def sentence2vector(sentence):
  vectors = []
  for word in sentence.split():
    if word in embeddings_index:
      vectors.append(embeddings_index[word])
    else:
      pass
  vectors = np.array(vectors)
  vector = np.mean(vectors, axis=0)
  return vector

def get_classes_map(dataframe):
  all_normal_rels = [eval(item)[1] for item in dataframe.normalized_triple.to_list()]
  rel2class = {}
  counter = 0
  for item in all_normal_rels:
    if item in rel2class:
      pass
    else:
      rel2class[item]=counter
      counter+=1
  # list(set(all_normal_rels))
  return rel2class
def get_frequent_rels(dataframe, count):
  all_normal_rels = [eval(item)[1] for item in dataframe.normalized_triple.to_list()]
  most_frequent_rels = [word for word, word_count in Counter(all_normal_rels).most_common(count)]
  return most_frequent_rels
def get_features(dataframe, most_frequent_rels):
  onehot_features = []
  for idx, row in tqdm(dataframe.iterrows()):
    temp_rel = eval(row.normalized_triple)[1]
    if temp_rel in most_frequent_rels:
      onehot_features.append(most_frequent_rels.index(temp_rel))
    else:
      onehot_features.append(len(most_frequent_rels))
  label_binarizer = LabelBinarizer()
  label_binarizer.fit(range(max(onehot_features)+1))
  onehot_features = label_binarizer.transform(onehot_features)
  embeddings_features = np.zeros((len(dataframe), 300), )
  for idx, row in tqdm(dataframe.iterrows()):
    vector = sentence2vector(row.Question)
    embeddings_features[idx, :] = vector
  features = np.hstack((embeddings_features, onehot_features))
  return np.nan_to_num(features)
def get_labels(dataframe, classes_map):
  labels = []
  for idx, row in tqdm(dataframe.iterrows()):
    temp_rel = eval(row.normalized_triple)[1]
    if temp_rel in classes_map:
      labels.append(classes_map[temp_rel])
    else:
      labels.append(-1)
  return labels

In [47]:
most_frequent_rels = get_frequent_rels(train, 300)
class_map = get_classes_map(train)
train_featurs = get_features(train, most_frequent_rels)
train_labels = get_labels(train, class_map)
valid_featurs = get_features(valid, most_frequent_rels)
valid_labels = get_labels(valid, class_map)
test_featurs = get_features(test, most_frequent_rels)
test_labels = get_labels(test, class_map)




9921it [00:01, 5916.19it/s]
393it [00:00, 3927.09it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
9921it [00:02, 4505.47it/s]
9921it [00:01, 6429.38it/s]
1751it [00:00, 5624.37it/s]
1751it [00:00, 4660.45it/s]
1751it [00:00, 6444.82it/s]
5003it [00:00, 5903.58it/s]
5003it [00:01, 4278.39it/s]
5003it [00:00, 6390.84it/s]


In [48]:
train_mask = np.array(train_labels)>=0
valid_mask = np.array(valid_labels)>=0
test_mask = np.array(test_labels)>=0
tsv_train = copy(train.loc[train_mask, ['Question']])
tsv_valid = copy(valid.loc[valid_mask, ['Question']])
tsv_test = copy(test.loc[test_mask, ['Question']])
tsv_train['rel_label'] = np.array(train_labels)[train_mask].tolist()
tsv_valid['rel_label'] = np.array(valid_labels)[valid_mask].tolist()
tsv_test['rel_label'] = np.array(test_labels)[test_mask].tolist()
tsv_train.to_csv('/content/drive/MyDrive/QA/train.tsv', sep = '\t', index=False)
tsv_valid.to_csv('/content/drive/MyDrive/QA/valid.tsv', sep = '\t', index=False)
tsv_test.to_csv('/content/drive/MyDrive/QA/test.tsv', sep = '\t', index=False)


In [49]:
from torchtext import data

class ReverbDataset(data.TabularDataset):
    @classmethod
    def splits(cls, text_field, label_field, path,
               train='train.tsv', validation='valid.tsv', test='test.tsv'):
        return super(ReverbDataset, cls).splits(
            path=path, train=train, validation=validation, test=test,
            format='tsv', fields=[('Question', text_field), ('rel_label', label_field)]
        )

In [ ]:
%cd /content/BuboQA/relation_prediction/nn
!python train.py  --fix_embed --relation_prediction_mode GRU \
                  --data_dir /content/drive/MyDrive/QA/ > gru.txt

/content/BuboQA/relation_prediction/nn
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)


In [ ]:
clf = LogisticRegression()
# print(valid_featurs.shape, train_featur)
X = np.vstack((train_featurs, valid_featurs)); y = np.array(train_labels+valid_labels)

mask = y >= 0
X = X[mask]; y=y[mask]

clf.fit(X, y)

X_test, y_test = test_featurs, np.array(test_labels)
mask = y_test >= 0
X_test = X_test[mask]; y_test=y_test[mask]


predicted_test = clf.predict(X_test)
accuracy_on_test = 100.0 * np.mean(predicted_test == y_test)
print("Accuracy on Test set: {}".format(accuracy_on_test))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy on Test set: 56.273199703043794
